In [2]:
# Loading modules
from PIL import Image, ImageTk
import tkinter as tk
import cv2
import os
import numpy as np
from keras.models import model_from_json
from string import ascii_uppercase

In [21]:
class Application:
    def __init__(self):
        
        # Initializing variables
        self.count=0
        self.sent_str=''
        self.word_str=''
        self.previous_let=''
        self.letter_list=["A",'B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S',' ','T','U','V','W','X','Y','Z' ]
        self.current_image = None
        self.current_image2 = None
        
        # Loading model
        #self.json_file = open("model_main.json", "r")
        #self.model_json = self.json_file.read()
        #self.json_file.close()
        #self.loaded_model = model_from_json(self.model_json)
        #self.loaded_model.load_weights("model_main.h5")
        #print("Loaded model from disk")
        
        # Initializing inerface
        self.root = tk.Tk()
        self.root.title("Sign language reader")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1500x950")
        self.panel = tk.Label(self.root)
        self.panel.place(x=60, y=20, width=720, height=720)
        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=900, y=140, width=310, height=310)
        self.label = tk.Label(self.root)
        self.label.place(x=250, y=17)
        self.label.config(text="Sign Language Reader",font=("Lucida Handwriting", 50, "bold"))
        self.panel3 = tk.Label(self.root)
        self.panel3.place(x=1100, y=505)
        self.label1 = tk.Label(self.root)
        self.label1.place(x=890, y=500)
        self.label1.config(text="Character :", font=("Courier", 30, "bold"))
        self.panel4 = tk.Label(self.root)
        self.panel4.place(x=1100, y=565)
        self.label2 = tk.Label(self.root)
        self.label2.place(x=890, y=560)
        self.label2.config(text="Word :", font=("Courier", 30, "bold"))
        self.panel5 = tk.Label(self.root)
        self.panel5.place(x=1100, y=625)
        self.label3 = tk.Label(self.root)
        self.label3.place(x=890, y=620)
        self.label3.config(text="Message :", font=("Courier", 30, "bold"))
        
        self.vs = cv2.VideoCapture(0)
        self.video_loop()
    def video_loop(self):
        ok, frame = self.vs.read()
        if ok:
            cv2image = cv2.flip(frame, 1)
            x1 = int(0.5*frame.shape[1])
            y1 = 10
            x2 = frame.shape[1]-10
            y2 = int(0.5*frame.shape[1])
            cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255, 0, 0), 1)
            cv2image = cv2.cvtColor(cv2image, cv2.COLOR_BGR2RGBA)
            self.current_image = Image.fromarray(cv2image)
            imgtk = ImageTk.PhotoImage(image=self.current_image)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)
            cv2image = cv2image[y1:y2, x1:x2]
            gray = cv2.cvtColor(cv2image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 2)
            th3 = cv2.adaptiveThreshold(
                blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
            ret, res = cv2.threshold(
                th3, 70, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            n=np.sum(res>0)
            #print(n)
            #self.panel3.config(text=str(n),font=("Courier",35,"bold"))
            """if n<94000:
                self.panel3.config(text="       ",font=("Courier",35,"bold"))
                out_temp=self.predict(res)
                self.panel3.config(text=self.letter_list[int(out_temp)],font=("Courier",35,"bold"))
                if self.previous_let==self.letter_list[int(out_temp)]:
                    self.count+=1
                else:
                    self.count=0
                    self.previous_let=self.letter_list[int(out_temp)]
                if self.count>15:
                    self.count=0
                    if self.letter_list[int(out_temp)]==' ':
                        self.sent_str=self.sent_str+" "+self.word_str
                        self.panel4.config(text="                      ",font=("Courier",35,"bold"))
                        self.word_str=''
                        self.panel5.config(text=self.sent_str,font=("Courier",35,"bold"))
                    else:
                        self.word_str=self.word_str+self.letter_list[int(out_temp)]
                        self.panel4.config(text=self.word_str,font=("Courier",35,"bold"))
            else:
                self.panel3.config(text="Blank",font=("Courier",35,"bold"))"""
            self.current_image2 = Image.fromarray(res)
            imgtk = ImageTk.PhotoImage(image=self.current_image2)
            self.panel2.imgtk = imgtk
            self.panel2.config(image=imgtk)
      
        self.root.after(30, self.video_loop)
    def predict(self,img):
        img = cv2.resize(img, (128, 128))
        result = self.loaded_model.predict(img.reshape(1, 128, 128, 1))
        return np.argmax(result)
    def destructor(self):
        print("Closing Application...")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()
        
sign_rec = Application()
sign_rec.root.mainloop()

Closing Application...
